<a href="https://colab.research.google.com/github/adithyaGHegde/Data-Science-ML-AI/blob/main/SentAnalysisIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gensim
import numpy as np
import nltk
from gensim import corpora, models, similarities
from nltk.corpus import stopwords
import re
from google.colab import files

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
reviews_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SentAnalysisIMDB/IMDB Dataset.csv')
reviews_df = pd.DataFrame(reviews_csv)
reviews_df = reviews_df.sample(n=20000)

In [ ]:
reviews_df_other = reviews_csv.drop(reviews_df.index)
print(reviews_df_other)

                                                  review sentiment
3      Basically there's a family where a little boy ...  negative
7      This show was an amazing, fresh & innovative i...  negative
8      Encouraged by the positive comments about this...  negative
9      If you like original gut wrenching laughter yo...  positive
11     I saw this movie when I was about 12 when it c...  negative
...                                                  ...       ...
49994  This is your typical junk comedy.<br /><br />T...  negative
49995  I thought this movie did a down right good job...  positive
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[30000 rows x 2 columns]


In [ ]:
print(reviews_df)

                                                  review sentiment
47223  Plotwise this is a rather silly little whodunn...  positive
23359  This was longer than the Ten Commandments, All...  negative
1130   I haven't had a chance to view the previous fi...  negative
10596  I never thought I see a movie worse than "Lisa...  negative
18486  This film is outstanding! On this date of APR,...  positive
...                                                  ...       ...
43932  Me and my friend read the summery and watched ...  negative
45133  When I saw the previews for this movie, I didn...  negative
28658  what a relief to find out I am not imagining t...  positive
6913   Shamefully, before I saw this film, I was unfa...  positive
19776  Cooley High is such a great film that even wit...  positive

[20000 rows x 2 columns]


In [ ]:
review_list = []
reviews = reviews_df_other['review'].values.tolist()
for review in reviews:
    cleanr = re.compile('<.*?>')
    review_clean = re.sub(cleanr, '', review)
    review_list.append(review_clean)

tok_corp = [nltk.word_tokenize(review) for review in review_list]

tok_refined = []
set_stopwords = set(stopwords.words('english'))
for tok_item in tok_corp:
    line_wo_stopwords  = [word for word in tok_item if word not in set_stopwords]
    line_refined = [word for word in line_wo_stopwords if word.isalpha()]
    tok_refined.append(line_refined)

model = gensim.models.Word2Vec(tok_refined, min_count=1)

total_len = int(len(tok_refined))
word_embeds = np.zeros(100 * total_len)
word_embeds = word_embeds.reshape(100, total_len)
print(word_embeds.shape)

embed_list = []
np.seterr(divide='ignore', invalid='ignore')

for i in range(total_len): 
  embed = np.zeros(100)
  for corp in tok_refined[i]:
      #     print(corp)
      ele_embed = np.copy(model.wv[corp])
      #     print(ele_embed, embed)
      embed += ele_embed
  embed_avg = np.divide(embed, len(tok_refined[i]))
  embed_list.append(embed_avg)
  # print(embed)
  word_embeds[:, i] = embed_avg

embeddings_test_df = pd.DataFrame(embed_list)
embeddings_test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/SentAnalysisIMDB/embeddings_test.csv')
print(embeddings_test_df.head())

X_final_test = embeddings_test_df
y_final_test = reviews_df_other['sentiment']
y_final_test.to_csv('/content/drive/MyDrive/Colab Notebooks/SentAnalysisIMDB/sentiment_test.csv')

y_final_test.replace('positive',1,inplace=True)
y_final_test.replace('negative',0,inplace=True)
y_final_test.reset_index(inplace = True, drop = True)
print(y_final_test.head())
print(X_final_test.head())

(100, 30000)
         0         1         2         3         4         5         6   \
0 -0.314510  0.001395 -0.123464 -0.148009  0.272739 -0.318705 -0.535717   
1 -0.844329 -0.270182 -0.136972 -0.586116  0.253170 -0.604329 -0.353819   
2 -0.699785 -0.131949 -0.151087 -0.344376  0.410121 -0.640867 -0.851077   
3 -0.359065  0.052118 -0.434849 -0.152588  0.228573 -0.082361 -0.399037   
4 -0.559737 -0.166035 -0.465889 -0.572349  0.287021 -0.329563 -0.632067   

         7         8         9   ...        90        91        92        93  \
0 -0.149990 -0.387238  0.876133  ... -0.027853 -0.326238  0.549185  0.004242   
1 -0.590307 -0.452114  0.604813  ... -0.066370 -0.208222  0.298994 -0.216115   
2 -0.747259 -0.329585  0.649563  ...  0.108098 -0.278959  0.295450 -0.060111   
3 -0.691023 -0.591687  0.794168  ... -0.411742 -0.181595  0.670089 -0.456801   
4 -0.614746 -0.450349  0.742722  ... -0.203922 -0.449108  0.551011 -0.132283   

         94        95        96        97        98    

In [ ]:
review_list = []
reviews = reviews_df['review'].values.tolist()
for review in reviews:
    cleanr = re.compile('<.*?>')
    review_clean = re.sub(cleanr, '', review)
    review_list.append(review_clean)

In [ ]:
print(review_list[:200])

["Plotwise this is a rather silly little whodunnit masquerading as a period drama/biopic.However the only reason I wanted to see it in the first place was because I was curious about what the great Henry Fonda was really like at his peak. I wasn't disappointed - he produces a truly warm and charismatic performance.In addition I can honestly say that I was never really bored at any stage during the film, so a strong ***1/2 Out of *****", "This was longer than the Ten Commandments, All Lord of the Rings and the Matrix Trilogy combined. My oh My, what a nightmare. This is the single biggest over-hype of 2006. THere is not a moment that is not scripted and clichéd. Movie Musicals can be done brilliantly and bring genuine excitement to the viewer. Dreamgirls takes the route of Chinese Water Torture, in the form of endless music montages, shoddy acting, and poor directing choices (Seriously, Mr. Condon, did you HAVE to do the old Billboard countdown shots? It's at #58! No wait, look its risi

In [ ]:
tok_corp = [nltk.word_tokenize(review) for review in review_list]

In [ ]:
tok_refined = []
set_stopwords = set(stopwords.words('english'))
for tok_item in tok_corp:
    line_wo_stopwords  = [word for word in tok_item if word not in set_stopwords]
    line_refined = [word for word in line_wo_stopwords if word.isalpha()]
    tok_refined.append(line_refined)

In [ ]:
print(tok_refined[:200])

[['Plotwise', 'rather', 'silly', 'little', 'whodunnit', 'masquerading', 'period', 'reason', 'I', 'wanted', 'see', 'first', 'place', 'I', 'curious', 'great', 'Henry', 'Fonda', 'really', 'like', 'peak', 'I', 'disappointed', 'produces', 'truly', 'warm', 'charismatic', 'addition', 'I', 'honestly', 'say', 'I', 'never', 'really', 'bored', 'stage', 'film', 'strong', 'Out'], ['This', 'longer', 'Ten', 'Commandments', 'All', 'Lord', 'Rings', 'Matrix', 'Trilogy', 'combined', 'My', 'oh', 'My', 'nightmare', 'This', 'single', 'biggest', 'THere', 'moment', 'scripted', 'clichéd', 'Movie', 'Musicals', 'done', 'brilliantly', 'bring', 'genuine', 'excitement', 'viewer', 'Dreamgirls', 'takes', 'route', 'Chinese', 'Water', 'Torture', 'form', 'endless', 'music', 'montages', 'shoddy', 'acting', 'poor', 'directing', 'choices', 'Seriously', 'Condon', 'HAVE', 'old', 'Billboard', 'countdown', 'shots', 'It', 'No', 'wait', 'look', 'rising', 'charts', 'passing', 'Billboard', 'notice', 'show'], ['I', 'chance', 'view'

In [ ]:
model = gensim.models.Word2Vec(tok_refined, min_count=1)

In [ ]:
total_len = int(len(tok_refined))
word_embeds = np.zeros(100 * total_len)
word_embeds = word_embeds.reshape(100, total_len)
print(word_embeds.shape)

(100, 20000)


In [ ]:
embed_list = []
np.seterr(divide='ignore', invalid='ignore')

for i in range(total_len): 
  embed = np.zeros(100)
  for corp in tok_refined[i]:
      #     print(corp)
      ele_embed = np.copy(model.wv[corp])
      #     print(ele_embed, embed)
      embed += ele_embed
  embed_avg = np.divide(embed, len(tok_refined[i]))
  embed_list.append(embed_avg)
  # print(embed)
  word_embeds[:, i] = embed_avg

In [ ]:
embeddings_df = pd.DataFrame(embed_list)
embeddings_df.to_csv('/content/drive/MyDrive/Colab Notebooks/SentAnalysisIMDB/embeddings.csv')
print(embeddings_df.head())

         0         1         2         3         4         5         6   \
0 -0.225335 -0.099113  0.008233  0.228292  0.418906  0.071753 -0.130993   
1 -0.058837 -0.212572 -0.027520 -0.163634  0.208851  0.063355 -0.019062   
2  0.054479 -0.216663 -0.087185  0.072038  0.123753  0.111433  0.053400   
3 -0.017656 -0.264215  0.106216 -0.021570  0.180616  0.078676 -0.005599   
4 -0.251473 -0.249904  0.225899 -0.036810  0.243959  0.170512 -0.023187   

         7         8         9   ...        90        91        92        93  \
0 -0.074555 -0.820417 -0.592425  ...  0.926169 -0.092960 -0.639846 -0.346367   
1 -0.293358 -0.524575 -0.473379  ...  0.670685  0.035209 -0.458917 -0.426942   
2 -0.213850 -0.657401 -0.592940  ...  0.889095  0.049677 -0.637866 -0.645227   
3 -0.308746 -0.573562 -0.535838  ...  0.712315  0.154803 -0.450633 -0.523111   
4 -0.265282 -0.666276 -0.678148  ...  0.815676 -0.010896 -0.424687 -0.469392   

         94        95        96        97        98        99  
0 -0

In [ ]:
X = embeddings_df

In [ ]:
embeddings_df.shape

(20000, 100)

In [ ]:
y = reviews_df['sentiment']
y.to_csv('/content/drive/MyDrive/Colab Notebooks/SentAnalysisIMDB/sentiment.csv')

In [ ]:
y.replace('positive',1,inplace=True)
y.replace('negative',0,inplace=True)
print(y.head())

47223    1
23359    0
1130     0
10596    0
18486    1
Name: sentiment, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Input

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.80)

In [ ]:
model_seq = Sequential()

In [ ]:
model_seq.add(Dense(45, activation='softmax', input_shape=(X_train.shape[1],)))
# model_seq.add(Dense(15,activation='softmax'))
model_seq.add(Dense(1,activation='sigmoid'))

In [ ]:
model_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 45)                4545      
                                                                 
 dense_1 (Dense)             (None, 15)                690       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 5,251
Trainable params: 5,251
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_seq.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
model_seq.fit(X_train, y_train, epochs=800, 
              validation_data=(X_test,y_test))

Epoch 1/800
125/125 [==============================] - 3s 23ms/step - loss: 0.4445 - acc: 0.8238 - val_loss: 1.5285 - val_acc: 0.5490
Epoch 2/800
125/125 [==============================] - 3s 23ms/step - loss: 0.4306 - acc: 0.8248 - val_loss: 1.4724 - val_acc: 0.5531
Epoch 3/800
125/125 [==============================] - 3s 23ms/step - loss: 0.4272 - acc: 0.8253 - val_loss: 1.5184 - val_acc: 0.5494
Epoch 4/800
125/125 [==============================] - 2s 15ms/step - loss: 0.4242 - acc: 0.8200 - val_loss: 1.5040 - val_acc: 0.5517
Epoch 5/800
125/125 [==============================] - 3s 23ms/step - loss: 0.4179 - acc: 0.8257 - val_loss: 1.5135 - val_acc: 0.5520
Epoch 6/800
125/125 [==============================] - 2s 15ms/step - loss: 0.4152 - acc: 0.8240 - val_loss: 1.4351 - val_acc: 0.5564
Epoch 7/800
125/125 [==============================] - 3s 22ms/step - loss: 0.4124 - acc: 0.8248 - val_loss: 1.4712 - val_acc: 0.5517
Epoch 8/800
125/125 [==============================] - 3s 23ms

In [ ]:
X_test = embeddings_test_df
y_test = reviews_df_other['sentiment']
# y_test.to_csv('/content/drive/MyDrive/Colab Notebooks/SentAnalysisIMDB/sentiment_test.csv')

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((X_final_test, y_final_test))
val_dataset = val_dataset.batch(64)

model_seq.fit(train_dataset, epochs=1, validation_data=val_dataset)

63/63 [==============================] - 2s 29ms/step - loss: 0.2261 - acc: 0.9130 - val_loss: 1.5329 - val_acc: 0.7100


In [ ]:
score = model_seq.evaluate(X_final_test, y_final_test, verbose=2) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])